In [ ]:
!pip install -q hidiffusion
!pip install -q diffusers==0.25.0 transformers==4.27.4 accelerate==0.18.0 xformers==0.0.16rc425

# Text-to-image task

## Initialize pipeline

In [ ]:
from hidiffusion import apply_hidiffusion, remove_hidiffusion
from diffusers import StableDiffusionXLPipeline, DDIMScheduler
import torch
pretrain_model = "stabilityai/stable-diffusion-xl-base-1.0"
scheduler = DDIMScheduler.from_pretrained(pretrain_model, subfolder="scheduler")
pipe = StableDiffusionXLPipeline.from_pretrained(pretrain_model, scheduler = scheduler, torch_dtype=torch.float16, variant="fp16").to("cuda")
pipe.enable_xformers_memory_efficient_attention()
pipe.enable_vae_tiling()

# Apply hidiffusion with a single line of code.

In [ ]:
# Apply hidiffusion with a single line of code.
apply_hidiffusion(pipe)

# Generate high-resolution image for visual enjoyment.

In [ ]:
prompt = "thick strokes, bright colors, an exotic fox, cute, chibi kawaii. detailed fur, hyperdetailed , big reflective eyes, fairytale, artstation,centered composition, perfect composition, centered, vibrant colors, muted colors, high detailed, 8k."
negative_prompt = "blurry, ugly, duplicate, poorly drawn, deformed, mosaic"
image = pipe(prompt, guidance_scale=7.5, height=2048, width=2048, eta=1.0, negative_prompt=negative_prompt).images[0]
image

# ControlNet task

## Initialize pipeline

In [ ]:
from diffusers import StableDiffusionXLControlNetPipeline, ControlNetModel, DDIMScheduler
import numpy as np
import torch
import cv2
from PIL import Image
from hidiffusion import apply_hidiffusion, remove_hidiffusion

# initialize the models and pipeline
controlnet_conditioning_scale = 0.5  # recommended for good generalization
controlnet = ControlNetModel.from_pretrained(
    "diffusers/controlnet-canny-sdxl-1.0", torch_dtype=torch.float16, variant="fp16"
)
scheduler = DDIMScheduler.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", subfolder="scheduler")
pipe = StableDiffusionXLControlNetPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", controlnet=controlnet, torch_dtype=torch.float16,
    scheduler = scheduler
)


# Apply hidiffusion with a single line of code.



In [ ]:
# Apply hidiffusion with a single line of code.
apply_hidiffusion(pipe)

In [ ]:
pipe.enable_vae_tiling()
pipe.enable_model_cpu_offload()
pipe.enable_xformers_memory_efficient_attention()

## Get canny images

In [ ]:
# load Yoshua_Bengio.jpg
path = './assets/Yoshua_Bengio.jpg'
image = Image.open(path)
# get canny image
image = np.array(image)
image = cv2.Canny(image, 100, 200)
image = image[:, :, None]
image = np.concatenate([image, image, image], axis=2)
canny_image = Image.fromarray(image)

# Generate high-resolution image for visual enjoyment.

In [ ]:
prompt = "a zombie, high face detail, high detail, muted color, 8k"
negative_prompt = "blurry, ugly, duplicate, poorly drawn, deformed, mosaic, poorly drawn teeth, duplicated teeth."
image = pipe(
    prompt, controlnet_conditioning_scale=controlnet_conditioning_scale, image=canny_image,
    height=2048, width=2048, guidance_scale=7.5, negative_prompt = negative_prompt, eta=1.0
).images[0]
image